In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor

# URL produk di Female Daily
base_url = "https://reviews.femaledaily.com/products/cleanser/toner/avoskin/avoskin-miraculous-refining-toner-1?page={}"

# Headers untuk menghindari pemblokiran
headers = {"User-Agent": "Mozilla/5.0"}

# List untuk menyimpan data (thread-safe)
list_review = []

# Fungsi untuk mengambil data dari satu halaman
def scrape_page(page):
    url = base_url.format(page)
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to fetch page {page}, status: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    reviews = soup.find_all("div", class_="jsx-2016320139 jsx-2462230538 item")

    page_reviews = []
    for review in reviews:
        try:
            user = review.find("p", class_="profile-username")
            username = user.text.strip() if user else "Unknown"

            rating = len(review.find_all("i", class_="icon-ic_big_star_full"))

            ulasan = review.find("p", class_="text-content")
            review_text = ulasan.text.strip() if ulasan else "No review"

            page_reviews.append({"Pengguna": username, "Rating": rating, "Ulasan": review_text})
        except AttributeError:
            continue  #Skip jika ada elemen yang tidak ditemukan

    print(f"Page {page} scraped, total reviews found: {len(page_reviews)}")
    return page_reviews

# Menjalankan scraping dengan threading
total_pages = 250  # Sesuaikan dengan kebutuhan
with ThreadPoolExecutor(max_workers=10) as executor:  # 20 thread berjalan bersamaan
    results = list(executor.map(scrape_page, range(1, total_pages + 1)))

# Menggabungkan hasil semua halaman
for res in results:
    list_review.extend(res)

# Simpan ke CSV
df = pd.DataFrame(list_review)
df.to_csv("ulasan_avoskin_threading.csv", index=False, encoding="utf-8")
print("Data berhasil disimpan ke ulasan_avoskin_threading.csv")